## Torchvision 객체 검출 미세조정 튜토리얼
* Penn-Fudan Database for Pedestrian Detection and Segmentation 데이터셋으로 미리 학습된 Mask R-CNN 모델을 미세조정하기

### 데이터셋 정의하기
* 객체 검출, 인스턴스 분할 및 사용자 키포인트 검출을 학습하기 위한 스크립트를 이용
* 데이터셋은 표준 `torch.utils.data.Dataset` 클래스를 상속받아야 하고
* `__len__`과 `__gettime__` 메소드를 구현해야 함
* 데이터터셋에서 필요한 유일한 특성은 `__gettime__`에서 다음을 반환해야 하는 것임:
    * 이미지
    * 대상
        * boxes: [x0, y0, x1, y1]
        * labels: 라벨정보, 0은 항상 배경 클래스
        * image_id: 이미지 구분자, 모든 이미지 간에 고유한 값이어야 함
        * area: 바운딩 박스의 면적
        * iscrowd: 참일 경우 평가에서 제외
        * mask(optional[N, H, W]): N개의 객체 마다의 마스크 정보
        * keypoints([N, K, 3]): N개의 객체마다 키포인트 정보, 키포인트는 [x, y, visibility]

### PennFudan를 위한 사용자 정의 데이터셋 작성하기



In [ ]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고, 정렬하여
        # 이미지와 분할 마스크 정렬을 확인합니다
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # 이미지와 마스크를 읽어옵니다
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # 분할 마스크는 RGB로 변환하지 않음을 유의하세요
        # 왜냐하면 각 색상은 다른 인스턴스에 해당하며, 0은 배경에 해당합니다
        mask = Image.open(mask_path)
        # numpy 배열을 PIL 이미지로 변환합니다
        mask = np.array(mask)
        # 인스턴스들은 다른 색들로 인코딩 되어 있습니다.
        obj_ids = np.unique(mask)
        # 첫번째 id 는 배경이라 제거합니다
        obj_ids = obj_ids[1:]

        # 컬러 인코딩된 마스크를 바이너리 마스크 세트로 나눕니다
        masks = mask == obj_ids[:, None, None]

        # 각 마스크의 바운딩 박스 좌표를 얻습니다
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # 모든 것을 torch.Tensor 타입으로 변환합니다
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 객체 종류는 한 종류만 존재합니다(역자주: 예제에서는 사람만이 대상입니다)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 모든 인스턴스는 군중(crowd) 상태가 아님을 가정합니다
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## 모델 정의하기
이번 튜토리얼에서는 Faster R-CNN에 기반한 Mask R-CNN 모델을 사용할 예정입니다

Mask R-CNN은 Faster R-CNN에 인스턴스에 대한 분할 마스크를 예측하는 추기 레이어를 추가한 모델입니다

Torchvision model zoo에서 사용 가능한 모델을 수정하는 방법은 크게 2가지 입니다
1. 미리 학습된 모델에서 마지막 레이어 수준만 미세 조정하는 것
2. 모델의 백본을 다른 백본으로 교체하는 것 (더 빠른 예측을 하려고 할 때 백본을 ResNet101 -> MobileNetV2로 교체)

### 1-미리 학습된 모델에서 마지막 레이어만 미세 조정하기
COCO 이미지로 사전에 학습된 모델에서 시작하여, 특정 클래스를 위해 미세 조정을 원한다고 가정해봅시다

In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# COCO로 미리 학습된 모델 읽기
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# 분류기를 새로운 것으로 교체하는데, num_classes는 사용자가 정의합니다
num_classes = 2  # 1 클래스(사람) + 배경
# 분류기에서 사용할 입력 특징의 차원 정보를 얻습니다
in_features = model.roi_heads.box_predictor.cls_score.in_features
# 미리 학습된 모델의 머리 부분을 새로운 것으로 교체합니다
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/uni/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

### 2-다른 백본을 추가하도록 모델 수정하기

In [2]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# Faster RNN은 백본의 출력 채널 수를 알아야 합니다
# mobilenetV2의 경우 1280이므로 여기에 추가해야 합니다
backbone.out_channels = 1280

# RPN(region proposal network)이 5개의 서로 다른 크기와 3개의 다른 측면 비율(Aspect ratio)를 가진
# 5 X 3개의 앵커를 공간 위치마다 생성하도록 합니다
# 각 feature map이 잠재적으로 다른 사이즈와 측면 비율을 가질 수 있기 때문에 Tuple[Tuple[int]] 타입을 가지도록 합니다

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))

# 관심 영역의 자르기 및 재할당 후 자르기 크기를 수행하는 데 사용할 feature map을 정의
# 만약 backbone이 tensor를 반환할 때, featmap_names는 [0]이 될 것으로 예상
# 일반적으로 backbone이 OrderedDict[Tensor] 타입을 반환해야 함
# feature map에서 사용할 featmap_names 값을 정할 수 있음

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)

# 조각들을 Faster RCNN 모델로 합칩니다
model = FasterRCNN(backbone, num_classes=2, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /home/uni/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]